In [1]:
#CODICE FUNZIONA PER DIVIDERE LE COLONNE E PRODURRE UN NUOVO FILE TXT
#funziona anche con catalogoNoHeader (ho tolto con un processsore per pdf
import fitz


CATALOGO = "CatalogoCutProva1NoHeader.pdf"
OUTPUT_FILE = "output.txt"

#Apre il file output con UTF-8 encoding
with open(OUTPUT_FILE, "w", encoding="utf-8") as file:

    
    #"text" è il parametro per l'OCR
    with fitz.open(CATALOGO) as doc:
        for page in doc:
            text = page.get_text("text")
            
            #Mette nel file
            file.write(text)

# Se funziona conferma 
print( f"funziona {OUTPUT_FILE}")


funziona output.txt


In [2]:
import re

# legge il txt come string
with open("output.txt", encoding="UTF-8") as f:
    corpora = f.read()


# toglie spazi, tabs e newline
re.sub('\s{2,}', ' ', corpora)
re.sub('\t+', ' ', corpora)
re.sub('\n+', '', corpora)


# toglie i numeri tra []
txt = re.split(r'\[\d*\s*\w*\s*[^\]]*\]', corpora)
#print(txt)


#splitta la stringa in due prima dell'informazione sul fondo di conservazione
result = []
for book in txt:
    BibInfo = re.split(r'\n(?=BTB)|\n(?=BAC)|\n(?=BAL)|\n(?=BTM)', book)
    
    BibInfo = [i for i in BibInfo if i] # remove empty strings
    #print(BibInfo)
    result.append(BibInfo)
    #print(result)

#toglie la prima list dove c'era VOLUMI ecc
del result[0]
#print(result)



# Extract the first sublists of each list using a list comprehension
#BOOKINFO contains info about the book: author, title, city, publisher and year
BookInfo = [lst[0] for lst in result]
#print(BookInfo)

# Extract the second sublists of each list using a list comprehension
#FUNDINFO CONTAINS THE INFO ABOUT THE CONSERVATION FUND
FundInfo = [sublist[1] for sublist in result if len(sublist) >= 2]
#print(FundInfo)

In [3]:
#toglie \n prima del nome se c'e
BookInfo = [s.strip() for s in BookInfo]

#print(BookInfo)

In [4]:
#TOGLIE LE FRASI CONTENTENTI A CURA DI E ANCHE LA FRASE DOPO PERCHE SPESSO E TIPO A. AMBROSINI.

import re


# Remove the sentences containing "A cura di" and the following sentences for each string
pattern = r'[^.]*[Aa]\s*cura\s*di[^.]*\.[^.]*\.'

result = []
for string in BookInfo:
    new_string = re.sub(pattern, '', string)
    result.append(new_string)

#print(result)

BookInfo = result
#print(BookInfo)

In [5]:
#TOGLIE LE FRASI CONTENTENTI TRADUZIONE DI E ANCHE LA FRASE DOPO PERCHE SPESSO E TIPO A. AMBROSINI.

import re


# Remove the sentences containing "A cura di" and the following sentences for each string
pattern = r'[^.]*[Tt]raduzion[^.]*\.[^.]*\.'

result = []
for string in BookInfo:
    new_string = re.sub(pattern, '', string)
    result.append(new_string)

#print(result)

BookInfo = result
#print(BookInfo)

In [6]:
#funziona per prendere autori,se c'è un - lo mette nei titoli

#<-----cosi  se c'è - scrive not found
#authors = []

#for item in BookInfo:
   # author = item.split('\n')[0]
    #if '-' in author:
       # authors.append('Not found')
    #else:
       # authors.append(author)

#print(authors)------>


#cosi invece crea dueliste, una titoli e una autori
authors = []

authors = []
titles =[]
for item in BookInfo:
    author = item.split('\n')[0]
    if '-' in author:
        titles.append(author)
    else:
        authors.append(author)
#print(authors)

    

In [7]:
#TOGLIE GLI ASTERISCHI PERCHE ERANO IN MEZZO AI COGLIONI
authors = [x.replace('*', '') for x in authors]

#print(authors)


In [8]:

# removing whitespace ad inizio e fine stringa
for i in range(len(authors)):
    authors[i] = authors[i].strip()

#print(authors)




#REPLACE ID CON IL NOME CHE LO PRECEDE


for i in range(len(authors)):
    if authors[i] == 'Id.' or authors[i] == 'Id':
        authors[i] = authors[i-1]

#print(authors)


In [9]:
#toglie i punti dalla lista di autori perchè quando togliebo authors da
#BookInfo toglieva tutti i punti
authorsNoDot = []  # Initialize an empty list to store the modified strings

# Iterate over each string in the 'authors' list
for string in authors:
    updated_string = string.replace(".", "")  # Remove periods from the current string
    authorsNoDot.append(updated_string)  # Append the modified string to the 'updated_strings' list
#print(authorsNoDot)
    
    
    
#toglie gli autori dalla vecchia lista
#toglie da BookInfo gli elementi della lista author 
#e crea una nuova lista TitleInfo con le informazioni rimaste
for i in range(len(BookInfo)):
    for substring in authorsNoDot:
        BookInfo[i] = BookInfo[i].replace(substring, "")
#print(BookInfo)

#TITLEINFO HA TITOLO, CITTA, CE, ANNO, ECC
TitleInfo = BookInfo
#print(TitleInfo)


In [10]:
#STAMPA LA LISTA DEGLI ANNI
#TOGLIE L'ANNO E IL RESTO DOPO L'ANNO

import re

def remove_after_year(list):
    pattern = r'\b(\d{4})\b'  # Regex pattern to match a 4-digit year surrounded by word boundaries
    years = []  # List to store the extracted years
    
    
    #TOGLIE L'ANNO E IL RESTO DOPO L'ANNO
    # Iterate over each string in the list
    for i in range(len(list)):
        match = re.search(pattern, list[i])  # Find the year match
        if match:
            year = match.group(1)  # Extract the year from the match
            years.append(year)  # Append the year to the years list
            
            year_index = match.start()  # Get the starting index of the year match
            list[i] = list[i][:year_index]  # Keep only the text before the year
    
    return list, years

# Example usage
result, years = remove_after_year(TitleInfo)
#print(result)
#print(years)
result = TitleInfo
#print(TitleInfo)


In [11]:
#TOGLIE LE \N VUOTE
def remove_newline_followed_by_comma(lst):
    pattern = r'\n,(?=\s*\S)'
    replacement = ','
    modified_lst = []
    for text in lst:
        modified_text = re.sub(pattern, replacement, text)
        modified_lst.append(modified_text.strip())
    return modified_lst

# Example usage
TitleInfo = remove_newline_followed_by_comma(TitleInfo)

#print(TitleInfo)


In [12]:
# Remove the trailing comma from each string
TitleInfo = [s.rstrip(',') for s in TitleInfo]


#PUBLISHER STANPA LA LISTA DI CASE EDITRICI
publisher = []


#TOGLIE LA CE DALLA STRING DI TITLEINFO
for item in TitleInfo:
    last_part = item.split(",")[-1].strip()  # Extract the last part and remove leading/trailing whitespace
    publisher.append(last_part)

TitleInfo = [item.replace(f", {last_part}", "") for item, last_part in zip(TitleInfo, publisher)]

#print(publisher)
#print(TitleInfo)


In [13]:
cities = []

for i in range(len(TitleInfo)):
    last_period_index = TitleInfo[i].rfind('.')
    if last_period_index != -1:
        city = TitleInfo[i][last_period_index + 1:].strip()
        cities.append(city)
        TitleInfo[i] = TitleInfo[i][:last_period_index].strip()

#print(cities)

title = TitleInfo
#print(title)


In [23]:
# Import pandas library
import pandas as pd
  
# initialize list elements
data = list(zip(authors, title, cities, publisher, years))

df = pd.DataFrame(data, columns=['Authors', 'Title', 'Cities', 'Publisher', 'Years'])
display(df)


df.to_csv('output.csv', index=False, encoding='utf-8')



df.to_csv('output.csv', sep=';')  # Replace ';' with the desired delimiter



,Authors,Title,Cities,Publisher,Years
0,"Abba, Giuseppe Cesare",Ricordi garibaldini,Torino,Sten,1913
1,"Abernethy, Silvia M.",. \nSomething to read. Qualcosa da \nleggere i...,Firenze,La \nNuova Italia,1946
2,"Abetti, Giorgio",Amici e nemici di Galileo,Mila-\nno,Bompiani,1945
3,"Abraham, Pierre",Créatures \nchez \nBalzac. \nRe-\ncherches sur...,Paris,NRF- Gallimard,1931
4,Accademia,Nazionale \ndei \nLincei \nAdunanza straordin...,Grosseto,Nazionale \ndei \nLincei \nAdunanza straordin...,1967
...,...,...,...,...,...
200,"Barbagallo, Corrado",Illusioni perdute,Milano,Garzan-\nti,1965
201,"Barbagallo, Corrado","Id. \nMercadet, homme d’affaires. Pa-\nris, Ni...",Milano,Marzo,1965
202,"Barbagallo, Corrado",Id. \nLes paysans: scènes de la vie de \ncampagne,Milano,Michel,1921
203,"Barbarani, Tiberio Umberto",Id. \nLe père Goriot. Introduzione e note \ndi...,Manuale di storia del Medioevo \n(dal 476 al,Cluny,1940


In [15]:
import pandas as pd

names = ['John', 'Emma', 'David']
ages = [25, 28, 22]
countries = ['USA', 'UK', 'Canada']

data = list(zip(names, ages, countries))

df = pd.DataFrame(data, columns=['Name', 'Age', 'Country'])

df.to_excel('output.xlsx', index=False)


df.to_csv('output.csv', sep=';')  # Replace ';' with the desired delimiter
